In [1]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import torch
# import torch.nn as nn
# from omegaconf import OmegaConf
from os.path import join
from os import listdir
import re
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
# files
var_vocab_filepath = "../../process_pipeline/data/builds/dx_250415_200_mean_clip/output/variables_vocabulary.json"
pos_vocab_filepath = "../../process_pipeline/data/builds/dx_250415_200_mean_clip/output/position_vocabulary.json"
rating_filepath = "../../process_pipeline/data/builds/dx_250415_200_mean_clip/control/steps_selected.xlsx"

with open(var_vocab_filepath, 'r', encoding='utf-8') as file:
    inv_var_dict = json.load(file)
    
with open(pos_vocab_filepath, 'r', encoding='utf-8') as file:
    inv_pos_dict = json.load(file)
    
# maps
var_map = {v: k[:3] for k, v in inv_var_dict.items()}
pos_map = {v: int(float(k)) for k, v in inv_pos_dict.items()}
rate_map = pd.read_excel(rating_filepath).set_index("Step")["Bewertung "].to_dict()

FileNotFoundError: [Errno 2] No such file or directory: '../../process_pipeline/data/builds/dx_250415_200_mean_clip/output/variables_vocabulary.json'

In [27]:
var_map

{0: 'las',
 1: 'las',
 2: 'las',
 3: 'las',
 4: 'las',
 5: 'las',
 6: 'las',
 7: 'las',
 8: 'gal',
 9: 'gal',
 10: 'gal',
 11: 'gal',
 12: 'gal',
 13: 'gal',
 14: 'gal',
 15: 'gal',
 16: 'gal',
 17: 'gal',
 18: 'gal',
 19: 'gal',
 20: 'gal',
 21: 'gal',
 22: 'gal',
 23: 'gal',
 24: 'gal',
 25: 'gal',
 26: 'gal',
 27: 'gal',
 28: 'gal',
 29: 'gal',
 30: 'gal',
 31: 'gal',
 32: 'gal',
 33: 'gal',
 34: 'gal',
 35: 'gal',
 36: 'gal',
 37: 'gal',
 38: 'gal',
 39: 'gal',
 40: 'gal',
 41: 'gal',
 42: 'gal',
 43: 'gal',
 44: 'gal',
 45: 'gal',
 46: 'gal',
 47: 'gal',
 48: 'gal',
 49: 'gal',
 50: 'gal',
 51: 'gal',
 52: 'gal',
 53: 'gal',
 54: 'gal',
 55: 'gal',
 56: 'gal',
 57: 'gal',
 58: 'gal',
 59: 'gal',
 60: 'gal',
 61: 'gal',
 62: 'gal',
 63: 'gal',
 64: 'gal',
 65: 'gal',
 66: 'gal',
 67: 'gal',
 68: 'gal',
 69: 'gal',
 70: 'gal',
 71: 'gal',
 72: 'gal',
 73: 'gal',
 74: 'gal',
 75: 'gal',
 76: 'gal',
 77: 'gal',
 78: 'gal',
 79: 'gal',
 80: 'gal',
 81: 'gal',
 82: 'gal',
 83: 'gal',
 8

In [5]:
experiment_dirpath = "../experiments/training/dx_250618_cat/emb20_mod20/sweeps/sweep_dec_pos_emb_hidden/sweep_dec_pos_emb_hidden_20/k_0/checkpoints/"
sweep_subpath = "" #"sweeps/sweep_optimization/sweep_optimization_4"
ckpt_subpath = "k_0/checkpoints/"

working_dirpath = experiment_dirpath#join(experiment_dirpath, sweep_subpath, ckpt_subpath)

regex_pattern = r"epoch=(\d+).*"

ckpt_dict = {}

for file in listdir(working_dirpath):
    match = re.search(regex_pattern, file)
    if match:
        file_path = join(working_dirpath, file)
        ckpt_dict[int(match.group(1))] = torch.load(file_path, map_location="cpu")["state_dict"]



print(ckpt_dict.keys())
ckpt_example = ckpt_dict[list(ckpt_dict.keys())[0]]


for k in ckpt_example.keys():
    print(k, ckpt_example[k].shape)

dict_keys([199, 299, 399, 499, 599, 699, 799, 899, 99, 999])
model.enc_embedding.embed_modules_list.0.embedding.embedding.weight torch.Size([400, 20])
model.enc_embedding.embed_modules_list.1.embedding.embedding.weight torch.Size([26, 20])
model.enc_embedding.embed_modules_list.2.embedding.embedding.weight torch.Size([20, 1])
model.enc_embedding.embed_modules_list.2.embedding.embedding.bias torch.Size([20])
model.enc_embedding.embed_modules_list.3.embedding.embed_weight torch.Size([5, 4])
model.enc_embedding.embed_modules_list.3.embedding.embed_bias torch.Size([5, 4])
model.dec_embedding.embed_modules_list.0.embedding.embedding.weight torch.Size([5, 20])
model.dec_embedding.embed_modules_list.2.embedding.embedding.weight torch.Size([20, 1])
model.dec_embedding.embed_modules_list.2.embedding.embedding.bias torch.Size([20])
model.dec_embedding.embed_modules_list.3.embedding.embed_weight torch.Size([5, 4])
model.dec_embedding.embed_modules_list.3.embedding.embed_bias torch.Size([5, 4])
mo

In [6]:
key_list = [
    "model.enc_embedding.embed_modules_list.0.embedding.embedding.weight",
    "model.enc_embedding.embed_modules_list.1.embedding.embedding.weight",
    "model.dec_embedding.embed_modules_list.0.embedding.embedding.weight",
    ]

In [7]:
ckpt_dict[999]

OrderedDict([('model.enc_embedding.embed_modules_list.0.embedding.embedding.weight',
              tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
                      [ 0.6541, -0.1949, -0.1492,  ...,  0.1385,  0.0050, -0.0605],
                      [ 0.3415,  0.0496, -0.0119,  ..., -0.1770,  0.1677, -0.1559],
                      ...,
                      [-0.2295, -0.0505,  0.0935,  ..., -0.0570, -0.2183,  0.2375],
                      [ 0.1604, -0.1880,  0.0108,  ..., -0.1005,  0.1652, -0.2653],
                      [ 0.0708,  0.6756, -0.1170,  ...,  0.0540,  0.0039,  0.0478]])),
             ('model.enc_embedding.embed_modules_list.1.embedding.embedding.weight',
              tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
                        0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
                        0.0000,  0.0000,  0.0000,  0.0000],
                      [-0.2829, -0.0201, -0.23

In [9]:
var_list = []
n_pca_comps = 3

full_dict = {}
for key in key_list:
    key_dict = {}
    
    for k in ckpt_dict.keys():
        epoch_dict = {}
        embeddings = ckpt_dict[k][key]
        pca = PCA(n_components=n_pca_comps)
        pcs = pca.fit_transform(embeddings)
        epoch_dict["explained_var"] = pca.explained_variance_ratio_
        epoch_dict["components"] = pcs
        key_dict[int(k)] = epoch_dict
    
    full_dict[key] = key_dict

pcs_var = full_dict[key_list[0]][999]["components"]

pcs_pos = full_dict[key_list[1]][999]["components"]


In [10]:
df_var = pd.DataFrame(pcs_var, columns=[f"PC{i}" for i in range(1, n_pca_comps + 1)])
df_var["process"]=df_var.index.map(var_map)

df_var_init = pd.DataFrame(pcs_var_init, columns=[f"PC{i}" for i in range(1, n_pca_comps + 1)])
df_var_init["process"]=df_var.index.map(var_map)

df_var

NameError: name 'var_map' is not defined

In [11]:
df = df_var_init
hue_label = "process"
df[hue_label] = df[hue_label].astype("category")
fig = px.scatter_3d(df, x='PC1', y='PC2', z='PC3', color=hue_label)
fig.show()


NameError: name 'df_var_init' is not defined

In [57]:
df = df_var
hue_label = "process"
df[hue_label] = df[hue_label].astype("category")
fig = px.scatter_3d(df, x='PC1', y='PC2', z='PC3', color=hue_label)
fig.show()

In [58]:
fig = make_subplots(
    rows=1, cols=2,
    shared_yaxes=True,          # keep scales aligned (optional)
    subplot_titles=("Initialization", "Trained")
)

# --- 3. Create px.scatter for each dataframe and graft its traces ---
scatter1 = px.scatter(df_var_init, x="PC1", y="PC2", color="process", height=400)
scatter1.update_traces(marker=dict(size=12))
for tr in scatter1.data:
    fig.add_trace(tr, row=1, col=1)       # left axis

scatter2 = px.scatter(df_var, x="PC1", y="PC2", color="process", height=400)
scatter2.update_traces(marker=dict(size=12))
for tr in scatter2.data:
    fig.add_trace(tr, row=1, col=2)       # right axis

# --- 4. Global cosmetics --------------------------------------------
fig.update_layout(
    template="plotly_white",
    width=950, height=450,
    legend_title_text="Group",
    margin=dict(l=60, r=30, t=60, b=40)
)

fig.show()

In [59]:
df_pos = pd.DataFrame(pcs_pos, columns=[f"PC{i}" for i in range(1, n_pca_comps + 1)])
df_pos["rating"]=df_pos.index.map(pos_map).map(rate_map)

df_pos_init = pd.DataFrame(pcs_pos_init, columns=[f"PC{i}" for i in range(1, n_pca_comps + 1)])
df_pos_init["rating"]=df_pos_init.index.map(pos_map).map(rate_map)

df_pos

,PC1,PC2,PC3,rating
0,0.049481,-0.028114,0.052205,3.0
1,-0.268882,0.148031,0.340030,3.0
2,0.849959,0.168690,0.283697,0.0
3,-0.638188,-0.095155,-0.255969,1.0
4,-0.520141,-0.405957,0.003726,3.0
5,-0.166843,0.648360,0.063253,0.0
6,-0.091109,-0.516524,0.180716,0.0
7,0.048220,-0.344518,0.574992,2.0
8,0.724592,0.077075,-0.311541,1.0
9,-0.348520,0.180007,0.032104,1.0


In [60]:
df = df_pos_init
df["rating"] = df["rating"].astype("category")
fig = px.scatter_3d(df, x='PC1', y='PC2', z='PC3', color='rating')
fig.show()

In [61]:
df = df_pos
df["rating"] = df["rating"].astype("category")
fig = px.scatter_3d(df, x='PC1', y='PC2', z='PC3', color='rating')
fig.show()

In [63]:
fig = make_subplots(
    rows=1, cols=2,
    shared_yaxes=True,          # keep scales aligned (optional)
    subplot_titles=("Initialization", "Trained")
)

# --- 3. Create px.scatter for each dataframe and graft its traces ---
scatter1 = px.scatter(df_pos_init, x="PC1", y="PC2", color="rating", height=400)
scatter1.update_traces(marker=dict(size=12))
for tr in scatter1.data:
    fig.add_trace(tr, row=1, col=1)       # left axis

scatter2 = px.scatter(df_pos, x="PC1", y="PC2", color="rating", height=400)
scatter2.update_traces(marker=dict(size=12))
for tr in scatter2.data:
    fig.add_trace(tr, row=1, col=2)       # right axis

# --- 4. Global cosmetics --------------------------------------------
fig.update_layout(
    template="plotly_white",
    width=950, height=450,
    legend_title_text="Group",
    margin=dict(l=60, r=30, t=60, b=40)
)

fig.show()